## Runs cross-instance quantitative analysis

In [ ]:
import dense_correspondence_manipulation.utils.utils as utils
import os

utils.add_dense_correspondence_to_python_path()
utils.set_cuda_visible_devices([1])

from dense_correspondence.evaluation.evaluation import DenseCorrespondenceEvaluation, DenseCorrespondenceEvaluationPlotter
from dense_correspondence.dataset.spartan_dataset_masked import SpartanDataset
import logging
logging.basicConfig(level=logging.INFO)

import pandas as pd
import numpy as np
import time

In [ ]:
eval_config_filename = os.path.join(utils.getDenseCorrespondenceSourceDir(), 'config', 'dense_correspondence',
                                    'evaluation',
                                    'evaluation.yaml')
eval_config = utils.getDictFromYamlFilename(eval_config_filename)
dce = DenseCorrespondenceEvaluation(eval_config)

In [ ]:
network_to_evaluate = 'shoes_actually_iterative'
cross_instance_labels = 'evaluation_labeled_data/shoe_annotated_keypoints.yaml'
full_path_cross_instance_labels = os.path.join(utils.getPdcPath(),cross_instance_labels)

network_path = eval_config["networks"][network_to_evaluate]["path_to_network_params"]
cross_instance_output_dir = os.path.join(os.path.dirname(utils.getDenseCorrespondenceSourceDir()),os.path.dirname(network_path),"analysis","cross_instance")
print cross_instance_output_dir

if not os.path.isdir(cross_instance_output_dir):
    os.makedirs(cross_instance_output_dir)
cross_instance_csv = os.path.join(cross_instance_output_dir, "data.csv")

In [ ]:
import time
start = time.time()
df = dce.evaluate_single_network_cross_instance(network_to_evaluate, full_path_cross_instance_labels)
print "took", time.time() - start, "seconds"

In [ ]:
print cross_instance_csv
df.to_csv(cross_instance_csv)

In [ ]:
DCEP = DenseCorrespondenceEvaluationPlotter
fig_axes = DCEP.run_on_single_dataframe(cross_instance_csv, label=network_to_evaluate, save=False)

In [ ]:
folder_name = "2018-10-15"
path_to_nets = os.path.join("code/data_volume/pdc/trained_models", folder_name)
path_to_nets = utils.convert_to_absolute_path(path_to_nets)
all_nets = sorted(os.listdir(path_to_nets))
nets_to_plot = []

nets_list = ["shoes_progress_actually_iterative_23", "shoes_3"]
for net in nets_list:
    nets_to_plot.append(os.path.join(folder_name,net))

In [ ]:
import matplotlib.pyplot as plt
dc_source_dir = utils.getDenseCorrespondenceSourceDir()

network_name = nets_to_plot[0]
path_to_csv = os.path.join(dc_source_dir, "data_volume", "pdc", "trained_models", network_name, "analysis/cross_instance/data.csv")
fig_axes = DCEP.run_on_single_dataframe(path_to_csv, label=network_name, save=False)

for network_name in nets_to_plot[1:]:
    path_to_csv = os.path.join(dc_source_dir, "data_volume", "pdc", "trained_models", network_name, "analysis/cross_instance/data.csv")
    fig_axes = DCEP.run_on_single_dataframe(path_to_csv, label=network_name, previous_fig_axes=fig_axes, save=False)

_, axes = fig_axes
axes[0].set_title("Cross instance")
plt.show()